# Tensorflow

- 구글에서 개발한 수치 계산과 대규모 머신러닝을 위한 오픈소스 라이브러리
- CPU/GPU 모드가 있는데, GPU 모드는 NVIDIA의 그래픽 카드가 없으면 사용할 수 없음
- GPU 모드는 CPU의 성능이 아닌 GPU의 성능이 중요

## 텐서플로우가 제공하는 것
- 핵심 구조는 numpy와 비슷, GPU를 지원
- (여러 장치와 서버에 대해)분산 컴퓨팅을 지원
- JIT 컴파일러 포함. 속도를 높이고 메모리 사용량을 줄이기 위해 계산 최적화
    - 파이썬 함수에서 계산 그래프를 추출한 후 최적화(가지치기) 효율적으로 실행. 
    - 독립적인 연산을 자동으로 병렬 실행
- 계산 그래프는 플랫폼에 중립적인 포맷으로 내보낼 수 있으므로 한 환경에서 텐스플로 모델을 훈련하고 다른 환경에서 실행 가능
- 텐서플로는 후진 모드 자동 미분 기능과 RMSProp, Nadam과 같은 고성능 옵티마이저를 제공하므로 모든 종류의 손실함수 쉽게 최소화

### 참고 사이트
- [TFX](https://tensorflow.org/tfx) 
- [텐서플로 모델 저장소](https://github.com/tensorflow/models) 
- [텐서플로 리소스 페이지](https://www.tensorflow.org/resources) : 텐서플로 기반 프로젝트
- [텐서플로 기반 프로젝트](https://github.com/jtoy/awesome-tensorflow)
- [스택오버플로](https://stackoverflow.com) : 기술적인 질문 시
- [텐서플로 깃허브](https://github.com/tensorflow/tensorflow) : 버그를 알리거나 새 기능 요청
- [텐서플로 포럼](https://discuss.tensorflow.org) : 일반적인 이야기

## Tensor
- Tensorflow의 기본 데이터 구조, 다차원 배열. 데이터는 텐서로 표현됨
- ndarray(numpy의 다차원 배열)
- 스칼라 값도 가질 수 있음(42 같은 단순한 값)
-사용자 정의 손실 함수, 사용자 정의 지표, 사용자 정의 층 등을 만들 때 중요. 



In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2919 - accuracy: 0.9152
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1423 - accuracy: 0.9574
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1052 - accuracy: 0.9684
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0876 - accuracy: 0.9728
Epoch 5/5
313/313 [==============================] - 0s 830us/step - loss: 0.0690 - accuracy: 0.9785


[0.06904041022062302, 0.9785000085830688]

# 2. 넘파이와 텐서플로
## 2.1. 텐서와 연산
- tf.constant()로 텐서 생성 가능, 2행 3열의 실수 행렬을 나타내는 텐서

In [3]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

- tf.Tensor는 크기(shape)와 데이터 타입(dtype)을 가짐

In [4]:
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
# 인덱스 참조
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis] # 1열을 추출

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

- 모든 종류의 텐서 연산이 가능

In [8]:
t + 10
# tf.add(t, 10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t) # 원소들을 제곱

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t) # 행렬곱 

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [11]:
import numpy as np
a = np.array([[2,3,4],[4,5,6]])
b = np.array([[1,3],[4,5],[6,7]])
a @ b

array([[38, 49],
       [60, 79]])

- 스칼라 값을 가지는 텐서, 이 경우 크기는 비어 있음

In [12]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

### 텐서플로가 제공하는 연산
#### 기본 수학 연산

- `tf.add()`
  - 두 텐서를 요소별로 더함
  - **Example:** `tf.add(x, y)`

- `tf.multiply()`
  - 두 텐서를 요소별로 곱함
  - **Example:** `tf.multiply(x, y)`

- `tf.square()`
  - 텐서의 각 요소를 제곱함
  - **Example:** `tf.square(x)`

- `tf.exp()`
  - 텐서의 각 요소에 대해 자연로그 밑 e의 지수 함수를 계산
  - **Example:** `tf.exp(x)`

- `tf.sqrt()`
  - 텐서의 각 요소에 대해 제곱근을 계산
  - **Example:** `tf.sqrt(x)`

#### numpy 제공 연산

- `tf.reshape()`
  - 텐서를 지정된 모양으로 재구성
  - **Example:** `tf.reshape(x, shape)`

- `tf.squeeze()`
  - 텐서에서 크기가 1인 차원을 제거
  - **Example:** `tf.squeeze(x)`

- `tf.tile()`
  - 텐서를 지정된 횟수만큼 반복
  - **Example:** `tf.tile(x, multiples)`

#### numpy와 이름이 다른 연산

- `tf.reduce_mean() = np.mean()`
  - 텐서의 요소들에 대한 평균을 계산
  - **Example:** `tf.reduce_mean(x)`

- `tf.reduce_sum() = np.sum()`
  - 텐서의 요소들에 대한 합을 계산
  - **Example:** `tf.reduce_sum(x)`

- `tf.reduce_max() = np.max()`
  - 텐서의 요소들 중 최대값을 계산
  - **Example:** `tf.reduce_max(x)`

- `tf.math.log() = np.log()`
  - 텐서의 각 요소에 대해 자연로그를 계산
  - **Example:** `tf.math.log(x)`

## 2.2. 텐서와 넘파이
- numpy 배열로 텐서를 만들 수 있고, 그 반대도 가능
- numpy 배열에 텐서플로 연산을 적용할 수 있고 텐서에 numpy 연산을 적용 가능

In [13]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [14]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## 2.3. 타입 변환
- 타입 변환은 성능 감소 우려
- 따라서 자동 타입 변환 X
- 호환되지 않는 타입의 텐서로 연산 실행 시 예외 발생
    - 실수 + 정수 텐서 연산 불가
    - 32비트 실수와 64비트 실수도 연산 불가

In [17]:
# 정수 + 실수 연산 불가
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [18]:
# 32비트와 64비트 연산 불가
tf.constant(2.) + tf.constant(40., dtype = tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

In [19]:
# 타입 변환
t2 = tf.constant(40., dtype = tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 2.4. 변수
- tf.Tensor : 변경 불가능한 객체
- tf.Variable : 변경 가능한 객체(신경망의 가중치를 저장, 시간에 따라 변경되는 파라미터에 사용)

In [20]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

- tf.Varaible은 tf.Tensor와 동일한 연산 가능
- numpy와 호환도 가능
- assign 메서드를 사용하여 변숫값을 바꿀 수 있음
- assign을 통하지 않은 직접 수정은 불가

In [21]:
v.assign(2 * v) # 바로 저장

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [22]:
v[0, 1].assign(42) # 바로 저장

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [23]:
v[:, 2].assign([0, 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [24]:
# 0행 0열과 1행 2열을 각각 100., 200.으로 변경
v.scatter_nd_update(indices = [[0, 0], [1, 2]], updates = [100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

## 2.5. 다른 데이터 구조
- 희소 텐서(tf.SparseTensor)
    - 대부분 0으로 채워진 텐서를 효율적으로 나타냄
    - tf.sparse패키지는 희소 텐서를 위한 연산을 제공
- 텐서 배열(tf.TensorArray)
    - 텐서의 리스트, 기본적으로 고정된 길이를 가지나 동적으로 바꿀 수 있음
    - 리스트에 포함된 모든 텐서는 크기와 데이터 타입이 동일해야 함
- 래그드 텐서(tf.RaggedTensor)
    - 리스트를 나타냄
    - 모든 텐서는 랭크와 데이터 타입이 같아야 하지만 크기는 다를 수 있음
    - 텐서의 크기가 달라지는 차원을 래그드 차원이라고 부름
    - tf.ragged 패키지는 래그드 텐서를 위한 연산을 제공
- 문자열 텐서
    - tf.string 타입의 텐서
    - 유니코드가 아닌 바이트 문자열을 나타냄
    - 자동으로 UTF-8로 인코딩
    - 유니코드 포인트를 나타내는 tf.int32 텐서를 사용해 유니코드 문자열을 표현 가능
    - tf.strings 패키지는 바이트 문자열, 유니코드 문자열과 이런 텐서 사이의 변환을 위한 연산을 제공
    - tf.string은 기본 데이터 타입이므로 문자열의 길이가 텐서 크기에 나타나지 않음
    - 유니코드 텐서로 바꾸면 문자열 길이가 텐서 크기에 표현
- 집합
    - 집합은 일반 텐서 혹은 희소텐서로 표현
    - `tf.constant([[1, 2], [3, 4]])`는 두 개의 집합 `{1, 2}`, `{3, 4}`를 나타냄
    - 각 집합은 텐서의 마지막 축에 있는 벡터에 의해 표현
    - tf.sets 패키지의 연산을 사용해 집합을 다룰 수 있음
- 큐
    - 단계별로 텐서를 저장
    - FIFOQueue : 일반적인 큐
    - PriorityQueue : 우선순위 큐
    - RandoShuffleQueue : 원소를 섞는 큐
    - PaddingFIFOQueue : 패딩을 추가해 크기가 다른 원소의 배치를 만드는 큐
    - tf.queue 패키지에 포함

# 3. 사용자 정의 모델과 훈련 알고리즘
## 3.1. 사용자 정의 손실 함수
- 후버 손실함수 구현
- 레이블과 모델의 예측을 매개변수로 받는 함수를 만들고 텐서플로 연산을 사용해 손실을 모두 담은 텐서를 계산

In [25]:
# 사용자 정의 후버 손실함수
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# model.compile(loss = huber_fn, optimizer = 'nadam')
# model.fit(X_train, y_train, [...])

## 3.2 사용자 정의 요소를 가진 모델을 저장하고 로드하기

In [ ]:
model = tf.keras.models.load_model('my_model_with_a_custom_loss', custom_objects = {'huber_fn' : huber_fn})

- 매개변수를 받을 수 있는 함수 적용

In [26]:
def create_huber(threshold = 1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < 1
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

# model.compile(loss=create_huber(2.0), optimezer = 'nadam')

- 모델 저장 시 threshold값은 저장 X, 로드 할 때 threshold 값을 지정해야 함

In [37]:
# model = tf.kereas.models.load_model('my_model_with_a_custom_loss_threshold_2', 
#                                     custom_objects = {'huber_fn' : create_huber(2.0)})

- tf.keras.losses.Loss 클래스를 상속하고 get_config() 메서드를 구현해 해결 가능

In [27]:
class HuberLoss(tf.keras.losses.Loss):
    # 부모 클래스의 생성자에 전달
    def __init__(self, threshold = 1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    
    # 레이블과 예측을 받고 모든 샘플의 손실을 계산해 반환
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error)  - self.threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    # 하이퍼파라미터 이름과 같이 매핑된 딕셔너리 반환
    # 먼저 부모 클래스의 get_config() 메서드 호출
    # 그 다음 딕셔너리에 새로운 파라미터 추가
    def get_config(self):
        base_config = super().get_config()
        return { **base_config, 'threshold' : self.threshold }

In [39]:
# model.compile(loss = HuberLoss(2.), optimizer='nadam')

- 모델 로드 시 클래스 이름과 클래스 자체를 매핑

In [40]:
# model = tf.keras.models.load_model('my_model_with_a_custom_loss_class', 
#                                   custom_objects = {'HuberLoss':HuberLoss})

## 3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

In [28]:
# 사용자 정의 활성화 함수
# tf.keras.activations.softplus() 와 동일
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

# 사용자 정의 글로럿 초기화
# tf.keras.initializers.glorat_normal() 과 동일
def my_glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

# 사용자 정의 L1 규제
# tf.keras.regularizers.l1(0.01)과 동일
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# 사용자 정의 제한(양수인 가중치만 남기는)
# tf.keras.constraints.nonneg()
# tf.nn.relu() 와 동일
def my_positive_weights(weights):
    return tf.where(weights < 0, tf.zeros_like(weights), weights)

- 층 예시

In [29]:
layer = tf.keras.layers.Dense(1, activation = my_softplus, 
                             kernel_initializer = my_glorot_initializer,
                             kernel_regularizer = my_l1_regularizer,
                             kernel_constraint = my_positive_weights)

- 이 활성화 함수는 Dense 층의 출력에 적용되고 그 다음 층에 그 결과가 전달
- 층의 가중치는 초기화 함수에서 반환된 값으로 초기화
- 훈련 스텝마다 가중치가 규제 함수에 전달되어 규제 손실을 계산하고 전체 손실에 추가되어 최종 손실 생성
- 제한 함수가 훈련 스텝마다 호출되어 층의 가중치를 제한한 가중치 값으로 바뀜

- 함수가 모델과 함께 저장해야 할 파라미터를 가지고 있는 경우 적절한 클래스를 상속한다.
- factor 하이퍼 파라미터를 저장하는 L1 규제를 위한 간단한 클래스의 예

In [30]:
class MyL1regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {'factor' : self.factor}

## 3.4. 사용자 정의 지표

### 손실과 지표의 차이
- 손실
    - 모델을 훈련하기 위해 경사 하강법에서 사용 및 미분 가능
    - 기울기가 모든 곳에서 0이 아니어야 함
- 지표
    - 모델 평가 시 사용
    - 모든 곳에서 기울기가 0이어도 괜찮음
    - 미분 불가능해도 됨
- 대부분의 경우 사용자 지표 함수를 만드는 것은 사용자 손실 함수를 만드는 것과 동일

In [44]:
# model.compile(loss = 'mse', optimizer = 'nadam', metrics = [create_huber(2.0)])

- 훈련하는 동안 각 배치에 대해 케라스는 지표를 계산하고 에포크가 시작할 때부터 평균을 기록
- 평균은 지표의 평균을 기록하는 것이 아닌 개수를 기록
- tf.keras.metrics.Precision 클래스가 하는 일

In [31]:
precision = tf.keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [32]:
# - 두 번째 배치의 정밀도가 아닌 누적 정밀도를 계산
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [33]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [34]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [35]:
precision.reset_states() # 변수 초기화

In [36]:
class HuberMetric(tf.keras.metrics.Metric):
    # 지표의 상태를 기록하기 위한 변수를 생성
    def __init__(self, threshold = 1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight('total', initializer='zeros') # 후버 손실의 합
        self.count = self.add_weight('count', initializer='zeros') # 처리한 샘플 수
    
    # 이 클래스를 함수처럼 사용할 때 호출
    # 배치의 레이블과 예측을 바탕으로 변수 업데이트
    def update_state(self, y_true, y_pred, sample_weight = None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    # 최종 결과를 계산하고 반환
    def result(self):
        return self.total / self.count
    
    # threshold 변수를 모델과 함께 저장
    def get_config(self):
        base_config = super().get_config()
        return { **base_config, "threshold" : self.threshold}

## 3.5 사용자 정의 층

In [38]:
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))